<a href="https://colab.research.google.com/github/RainbowPowerr/Are-AI-photographers-ready-for-hire/blob/main/Generating_the_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stable diffusion**

Set up:

In [ ]:
!nvidia-smi

!pip install diffusers==0.8.0
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

from google.colab import output
output.enable_custom_widget_manager()

from huggingface_hub import notebook_login

notebook_login()

Download model and prep

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16")
pipe = pipe.to("cuda")

from PIL import Image

import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

# Generate the images 



In [ ]:
import pandas as pd

df = pd.read_csv(r'/content/image_data_with_index.csv')

print(df.head())

In [ ]:
def extra(xtra_prompt): 
  if xtra_prompt == 1: extra_words = ", cinematic, colorful background, high detail, highly detailed, hyper realistic, intricate, intricate sharp details, smooth, studio lighting"
  else: extra_words = " "
  return extra_words


def main(prompt):
  if prompt == "houseing": main_prompt =  " straight view of a tiny red house next to a stack of golden coins, 4k, hd"
  elif prompt == "weather": main_prompt =  " a forest in Sweden by a lake, autumn, sun is shining, 4k, hd"
  elif prompt == "bankruptcy": main_prompt =  " cardboard boxes and plants, on a table, in an office, folders, sideview, 4k, hd"
  else: main_prompt = " "
  return main_prompt

In [ ]:
createFolder('./Images/Main_Images/')

num_images = len(df)

for i in range(num_images):
  generator = torch.Generator("cuda").manual_seed(6596 + int(df["index"][i]))
  with open("log.txt", "w") as f:
    f.write(f"Iteration {i}")
  prompt = df["prompt"][i]
  xtra_prompt = df["xtra_prompt"][i]
  images = pipe(prompt=(main(prompt)+extra(xtra_prompt)), num_inference_steps=df["inf_steps"][i], guidance_scale = df["guidance"][i], height=400 , width=720, generator=generator ).images
  images[0].save(f'./Images/Main_Images/00{df["index"][i]}.png')

Create a zip-file to download the images

In [ ]:
!zip -r /content/Images.zip /content/Images